In [1]:
from deap import base, creator, tools, algorithms
import numpy as np
import random
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
from tqdm.notebook import tqdm

In [2]:
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-small")
model = AutoModel.from_pretrained("facebook/dinov2-small")
model = model.to("mps")

In [3]:
target_img = Image.open("./grant-headshot.png")

In [4]:
def get_img_vects(imgs):
    inputs = processor(images=imgs, return_tensors="pt")
    inputs = {k: v.to("mps") for k, v in inputs.items()}
    outputs = model(**inputs)
    # vects = outputs.last_hidden_state.mean(axis=1).detach().cpu().numpy()
    vects = outputs.last_hidden_state[:, 1:, :].detach().cpu().numpy()
    # normalize
    # vects /= np.linalg.norm(vects, axis=1, keepdims=True)
    return vects

In [5]:
target_vect = get_img_vects(target_img)[0]

/opt/homebrew/lib/python3.11/site-packages/torch/nn/functional.py:3967: UserWarning: The operator 'aten::upsample_bicubic2d.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  return torch._C._nn.upsample_bicubic2d(input, output_size, align_corners, scale_factors)


In [6]:
# Parameters
IMG_SIZE = 32
POPULATION_SIZE = 300
MUTATION_RATE = 0.05
CROSSOVER_RATE = 0.5
N_GENERATIONS = 100

# Target vector for similarity comparison
# target_vect = np.random.rand(224, 224, 3)  # Placeholder for your target image vector

def evaluate(individual):
    # Convert individual to image
    img = Image.fromarray(np.array(individual, dtype=np.uint8).reshape(IMG_SIZE, IMG_SIZE) * 255).convert("RGB")
    # Calculate similarity to target image (placeholder function)
    img_vect = get_img_vects(img)[0]
    similarity = ((img_vect - target_vect) ** 2).mean()
    return (-similarity,)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, IMG_SIZE * IMG_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=MUTATION_RATE)
toolbox.register("select", tools.selTournament, tournsize=3)

# Initialize population
population = toolbox.population(n=POPULATION_SIZE)

# Algorithm
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

population, logbook = algorithms.eaSimple(population, toolbox, cxpb=CROSSOVER_RATE, mutpb=MUTATION_RATE, ngen=N_GENERATIONS, stats=stats, verbose=True)


gen	nevals	avg     	min     	max     
0  	300   	-11.9686	-12.4772	-11.5667
1  	277   	-11.8512	-12.2732	-11.4523
2  	262   	-11.7665	-12.2267	-11.3143
3  	275   	-11.6846	-12.1138	-11.1242
4  	267   	-11.61  	-12.1289	-11.0537
5  	275   	-11.5316	-12.1339	-11.0211
6  	270   	-11.4613	-12.0091	-10.8911
7  	263   	-11.3941	-11.941 	-10.8902
8  	278   	-11.3386	-11.7878	-10.8368
9  	267   	-11.3041	-11.9525	-10.7271
10 	250   	-11.2372	-11.7918	-10.6872
11 	269   	-11.1829	-11.7933	-10.6246


KeyboardInterrupt: 